# Module 5 - Programming Assignment

## Directions

1. Change the name of this file to be your JHED id as in `jsmith299.ipynb`. Because sure you use your JHED ID (it's made out of your name and not your student id which is just letters and numbers).
2. Make sure the notebook you submit is cleanly and fully executed. I do not grade unexecuted notebooks.
3. Submit your notebook back in Blackboard where you downloaded this file.

*Provide the output **exactly** as requested*

## Solving Normal Form Games

In [1]:
from typing import List, Tuple, Dict, Callable
from copy import deepcopy

In the lecture we talked about the Prisoner's Dilemma game, shown here in Normal Form:

Player 1 / Player 2  | Defect | Cooperate
------------- | ------------- | -------------
Defect  | -5, -5 | -1, -10
Cooperate  | -10, -1 | -2, -2

where the payoff to Player 1 is the left number and the payoff to Player 2 is the right number. We can represent each payoff cell as a Tuple: `(-5, -5)`, for example. We can represent each row as a List of Tuples: `[(-5, -5), (-1, -10)]` would be the first row and the entire table as a List of Lists:

In [2]:
prisoners_dilemma = [
 [( -5, -5), (-1,-10)],
 [(-10, -1), (-2, -2)]]

prisoners_dilemma

[[(-5, -5), (-1, -10)], [(-10, -1), (-2, -2)]]

in which case the strategies are represented by indices into the List of Lists. For example, `(Defect, Cooperate)` for the above game becomes `prisoners_dilemma[ 0][ 1]` and returns the payoff `(-1, -10)` because 0 is the first row of the table ("Defect" for Player 1) and 1 is the 2nd column of the row ("Cooperate" for Player 2).

For this assignment, you are going write a function that uses Successive Elimination of Dominated Strategies (SEDS) to find the **pure strategy** Nash Equilibrium of a Normal Form Game. The function is called `solve_game`:

```python
def solve_game( game: List[List[Tuple]], weak=False) -> List[Tuple]:
    pass # returns strategy indices of Nash equilibrium or None.
```

and it takes two parameters: the game, in a format that we described earlier and an optional boolean flag that controls whether the algorithm considers only **strongly dominated strategies** (the default will be false) or whether it should consider **weakly dominated strategies** as well.

It should work with game matrices of any size and it will return the **strategy indices** of the Nash Equilibrium. If there is no **pure strategy** equilibrium that can be found using SEDS, return the empty List (`[]`).


<div style="background: mistyrose; color: firebrick; border: 2px solid darkred; padding: 5px; margin: 10px;">
Do not return the payoff. That's not useful. Return the strategy indices, any other output is incorrect.
</div>

As before, you must provide your implementation in the space below, one Markdown cell for documentation and one Code cell for implementation, one function and assertations per Codecell.


---

## `Check Strong Domination` <a id="strong_domination"></a>

**Description:**
This function determines whether one strategy strongly dominates another. Strong domination occurs when every element of the first strategy is less than the corresponding element of the second strategy, indicating a more preferable outcome in every scenario. This is necessary for our algorithm to work as it enables it to narrow down possible Nash Equilibriums.

**Parameters:**
- `strategy` (`List[int]`): A list representing a strategy with integer payoffs.
- `other_strategy` (`List[int]`): Another

**Returns:**
- `strong_domination` (`Bool`): Returns `True` if the first strategy strongly dominates the second, otherwise `False`.

In [3]:
def check_strong_domination(strategy: List[int], other_strategy: List[int]) -> bool:
    strong_domination = all(a < b for a, b in zip(strategy, other_strategy))
    return strong_domination

In [4]:
assert check_strong_domination([1, 2, 3], [2, 3, 4]) == True #Second Column should dominate first
assert check_strong_domination([1, 2, 4], [2, 3, 4]) == False # Not a Strong Domination as 4 = 4
assert check_strong_domination([2, 3, 4], [2, 3, 4]) == False # Not a Strong Domination as they are equal

### Check Weak Domination <a id="weak_domination"></a>

**Description:**
This function checks if one strategy weakly dominates another. Weak domination means that all elements of the first strategy are at least as good as the corresponding elements in the second strategy, with at least one element being strictly better and no elements being worse. This is necessary for our algorithm to work as it enables it to narrow down possible Nash Equilibriums.

**Parameters:**
- `strategy` (`List[int]`): A list representing a strategy with integer payoffs, where lower values indicate better outcomes.
- `other_strategy` (`List[int]`): A list against which the first strategy is compared.

**Returns:**
- `bool`: Returns `True` if the first strategy weakly dominates the second, otherwise `False`.


In [5]:
def check_weak_domination(strategy: List[int], other_strategy: List[int]) -> bool:
    weak_domination = (any(a < b for a, b in zip(strategy, other_strategy)) and all(a <= b for a, b in zip(strategy, other_strategy)))
    return weak_domination

In [6]:
assert check_weak_domination([1, 2, 3], [1, 2, 4]) == True # Should be weakly dominated
assert check_weak_domination([1, 3, 4], [1, 2, 4]) == False # Not weakly dominated as it checks 2nd to 1st
assert check_weak_domination([2, 3, 4], [2, 3, 4]) == False # All equal is not a weak domination

### Remove Dominated Rows <a id="remove_dominated_rows"></a>

**Description:**
This function removes dominated rows from the game matrix. This process simplifies the game matrix by reducing the number of strategies, making the analysis more manageable.

**Parameters:**
- `game` (`List[List[Tuple[int, int]]]`): The game matrix, where each entry is a tuple representing the payoffs for two players.
- `remaining_rows` (`List[int]`): List of indices of the rows that are still in the game.
- `n` (`int`): The current number of active rows in the game matrix.
- `m` (`int`): The number of columns in the game matrix.
- `weak` (`bool`): A flag indicating whether to use weak domination (True) or strong domination (False) for elimination.

**Returns:**
- `Tuple[List[List[Tuple[int, int]]], List[int], int]`: A tuple containing the updated game matrix, the list of remaining row indices, and the new count of rows. All information is needed to ensure the solve_game returns original game board positions instead of reduced positions.


In [7]:
def remove_dominated_rows(game: List[List[Tuple[int, int]]], remaining_rows: List[int], n: int, m: int, weak: bool) -> Tuple[List[List[Tuple[int, int]]], List[int], int]:
    row_change = True
    while row_change:
        row_change = False
        for i in range(n):
            for j in range(n):
                if i != j:
                    current_row = [game[i][k][0] for k in range(m)]
                    other_row = [game[j][k][0] for k in range(m)]
                    if (weak and check_weak_domination(current_row, other_row)) or (not weak and check_strong_domination(current_row, other_row)):
                        game.pop(i)
                        remaining_rows.pop(i)
                        n -= 1
                        row_change = True
                        break
            if row_change:
                break
    return game, remaining_rows, n

In [8]:
# Example game matrix for testing
game_example = [
    [(10, 10), (10, 6)],
    [(2, 5), (3, 6)],
    [(1, 4), (2, 7)]
]

remaining_rows_example = [0, 1, 2]
game_updated, remaining_rows_updated, n_updated = remove_dominated_rows(game_example, remaining_rows_example, 3, 2, False)
assert game_updated == [[(10, 10), (10, 6)]] # All rows except row 1 should be removed as the are strongly dominated

game_example = [
    [(10, 10), (10, 6)],
    [(2, 5), (10, 6)],
    [(1, 4), (2, 7)]
]
remaining_rows_example = [0, 1, 2]
game_updated, remaining_rows_updated, n_updated = remove_dominated_rows(game_example, remaining_rows_example, 3, 2, True)
assert game_updated == [[(10, 10), (10, 6)]] # All rows but first one shold be removed as they are weakly dominated

game_example = [
    [(1, 5), (2, 6)],
    [(2, 5), (1, 6)],
    [(1, 4), (2, 7)]
]
remaining_rows_example = [0, 1, 2]
game_updated, remaining_rows_updated, n_updated = remove_dominated_rows(game_example, remaining_rows_example, 3, 2, False)
assert game_updated == [
    [(1, 5), (2, 6)],
    [(2, 5), (1, 6)],
    [(1, 4), (2, 7)]
] # No row sould be removed

### Remove Dominated Columns <a id="remove_dominated_cols"></a>

**Description:**
This function removes dominated cols from the game matrix. This process simplifies the game matrix by reducing the number of strategies, making the analysis more manageable.

**Parameters:**
- `game` (`List[List[Tuple[int, int]]]`): The game matrix, where each entry is a tuple representing the payoffs for two players.
- `remaining_rows` (`List[int]`): List of indices of the cols that are still in the game.
- `n` (`int`): The current number of active cols in the game matrix.
- `m` (`int`): The number of columns in the game matrix.
- `weak` (`bool`): A flag indicating whether to use weak domination (True) or strong domination (False) for elimination.

**Returns:**
- `Tuple[List[List[Tuple[int, int]]], List[int], int]`: A tuple containing the updated game matrix, the list of remaining col indices, and the new count of cols. All information is needed to ensure the solve_game returns original game board positions instead of reduced positions.


In [9]:
def remove_dominated_columns(game: List[List[Tuple[int, int]]], remaining_cols: List[int], n: int, m: int, weak: bool) -> Tuple[List[List[Tuple[int, int]]], List[int]]:
    col_change = True
    while col_change:
        col_change = False
        m = len(game[0]) if game else 0
        for i in range(m):
            for j in range(m):
                if i != j:
                    current_col = [game[k][i][1] for k in range(n)]
                    other_col = [game[k][j][1] for k in range(n)]
                    if (weak and check_weak_domination(current_col, other_col)) or (not weak and check_strong_domination(current_col, other_col)):
                        for row in game:
                            row.pop(i)
                        remaining_cols.pop(i)
                        col_change = True
                        break
            if col_change:
                break
    return game, remaining_cols

In [10]:
game_example = [
    [(10, 10), (2, 5), (1, 4)],
    [(10, 10), (10, 6), (2, 7)]
]

remaining_cols_example = [0, 1, 2]
game_updated, remaining_cols_updated = remove_dominated_columns(game_example, remaining_cols_example, 2, 3, False)
assert game_updated == [[(10, 10)], [(10, 10)]]  # Columns 1 and 2 should be removed as they are strongly dominated by column 0

game_example = [
    [(1, 10), (2, 2), (3, 1)],
    [(2, 10), (1, 10), (4, 2)]
]
remaining_cols_example = [0, 1, 2]
game_updated, remaining_cols_updated = remove_dominated_columns(game_example, remaining_cols_example, 2, 3, True)
assert game_updated == [[(1, 10)], [(2, 10)]]  # Columns 1 and 2 should be removed as they are weakly dominated by column 0

game_example = [
    [(1, 5), (2, 5), (3, 4)],
    [(2, 5), (1, 6), (4, 7)]
]
remaining_cols_example = [0, 1, 2]
game_updated, remaining_cols_updated = remove_dominated_columns(game_example, remaining_cols_example, 2, 3, False)
assert game_updated == [
    [(1, 5), (2, 5), (3, 4)],
    [(2, 5), (1, 6), (4, 7)]
]  # No column should be removed as no strong dominance is present


### Eliminate Dominated Strategies <a id="eliminate_dominated_strategies"></a>

**Description:**
The function removes rows and columns via whichever domination strategy we are utilizing. It is necessary for the solve game function as it enables us to know what columns and rows should be eliminated to narrow down our search.

**Parameters:**
- `game` (`List[List[Tuple[int, int]]]`): The game matrix, where each entry is a tuple representing the payoffs for two players.
- `weak` (`bool`): A flag that indicates whether to use weak domination (`True`) or strong domination (`False`) for the elimination process.

**Returns:**
- `Tuple[List[List[Tuple[int, int]]], List[int], List[int]]`: A tuple containing the updated game matrix, the list of remaining row indices, and the list of remaining column indices.


In [11]:
def eliminate_dominated_strategies(game: List[List[Tuple[int, int]]], weak: bool = False) -> Tuple[List[List[Tuple[int, int]]], List[int], List[int]]:
    game_copy = deepcopy(game)
    n = len(game_copy)
    m = len(game_copy[0]) if game_copy else 0

    remaining_rows = list(range(n))
    remaining_cols = list(range(m))

    game_copy, remaining_rows, n = remove_dominated_rows(game_copy, remaining_rows, n, m, weak)
    game_copy, remaining_cols = remove_dominated_columns(game_copy, remaining_cols, n, m, weak)

    return game_copy, remaining_rows, remaining_cols


In [12]:
game_example = [
    [(3, 1), (2, 6), (1, 2)],
    [(2, 1), (4, 6), (1, 3)],
    [(3, 1), (2, 6), (0, 2)]
]

eliminated_game, rows_remaining, cols_remaining = eliminate_dominated_strategies(game_example, weak=False)
assert len(eliminated_game[0]) == 1 # The side two columns should be dominated by the middle column, so only 1 column should remain

eliminated_game_weak, rows_remaining_weak, cols_remaining_weak = eliminate_dominated_strategies(game_example, weak=True)
assert len(eliminated_game_weak) < len(eliminated_game) # Weak domination will remove another position compared to Strong domination 

assert len(cols_remaining) < len(game_example[0]) # Columns should be removed from this example

### Is Nash Equilibrium <a id="is_nash_equilibrium"></a>

**Description:**
This function checks if a specified pair of strategies in a game matrix is a Nash equilibrium. A Nash equilibrium occurs when each player's strategy is the best response to the other's, meaning no player can benefit by unilaterally changing their strategy. Required to "solve" the game.

**Parameters:**
- `game` (`List[List[Tuple[int, int]]]`): The game matrix where each cell contains a tuple representing the payoffs for Player 1 and Player 2, respectively.
- `p1` (`int`): The index of Player 1's strategy.
- `p2` (`int`): The index of Player 2's strategy.

**Returns:**
- `bool`: Returns `True` if the (p1, p2) strategy pair is a Nash equilibrium, otherwise `False`.


In [13]:
def is_nash_equilibrium(game: List[List[Tuple[int, int]]], p1: int, p2: int) -> bool:
    p1_payoff = game[p1][p2][0]
    p2_payoff = game[p1][p2][1]

    # Check if Player 1's strategy is the best response to Player 2's strategy
    if any(game[i][p2][0] > p1_payoff for i in range(len(game))):
        return False
    
    # Check if Player 2's strategy is the best response to Player 1's strategy
    if any(game[p1][j][1] > p2_payoff for j in range(len(game[0]))):
        return False

    return True

In [14]:
game_example = [
    [(3, 3), (1, 2)],
    [(2, 1), (0, 0)]
]

assert is_nash_equilibrium(game_example, 0, 0) == True # (0, 0) is nash equilibrium
assert is_nash_equilibrium(game_example, 1, 1) == False # (1, 1) is not a nash equilibrium
assert is_nash_equilibrium(game_example, 0, 1) == False # (0, 1) is not a nash equilibrium

## solve_game <a id="solve_game"></a>

**Description:**
This function simplifies a game matrix by eliminating dominated strategies and then checks for Nash equilibria in the reduced game matrix. A boolean determining the use of strong or weak domination can be passed in.

**Parameters:**
- `game` (`List[List[Tuple[int, int]]]`): The game matrix, where each cell contains a tuple representing the payoffs for Player 1 and Player 2, respectively.
- `weak` (`bool`): A flag indicating whether to use weak domination (`True`) or strong domination (`False`) for the elimination process.

**Returns:**
- `List[Tuple[int, int]]`: A list of tuples where each tuple represents a pair of strategy indices `(row_index, col_index)` that form a Nash equilibrium in the original game matrix.

In [15]:
def solve_game(game: List[List[Tuple[int, int]]], weak: bool = False) -> List[Tuple[int, int]]:
    reduced_game, remaining_rows, remaining_cols = eliminate_dominated_strategies(game, weak)
    
    # If nothing reduced we could techinally use State Space Search, but then its not really a SEDS or SEWDS solve
    if reduced_game == game:
        return []

    # Now, check for Nash equilibria in the reduced game
    equilibria = []
    for p1 in range(len(reduced_game)):
        for p2 in range(len(reduced_game[0])):
            if is_nash_equilibrium(reduced_game, p1, p2):
                # Map the reduced game indices back to the original game indices
                equilibria.append((remaining_rows[p1], remaining_cols[p2]))

    return equilibria

In [17]:
prisoners_dilemma = [
    [(-5, -5), (-1, -10)],
    [(-10, -1), (-2, -2)]
]

matching_pennies = [
    [(1, -1), (-1, 1)],
    [(-1, 1), (1, -1)]
]

SEWDS_game = [
    [(2, 2), (1, 4)],
    [(2, 3), (4, 1)]
]

assert(solve_game(prisoners_dilemma, weak=False) == [(0,0)]) #Returns first choice as it is best
assert(solve_game(SEWDS_game, weak=True) == [(1, 0)]) # Can get with SEWDS but not SEDS
assert(solve_game(matching_pennies, weak=False) == [])# Returns nothing as there is not a pure equilibria


## Additional Directions

Create three games as described and according to the following:

1. Your games must be created and solved "by hand".
2. The strategy pairs must **not** be on the main diagonal (0, 0), (1, 1), or (2, 2). And the solution cannot be the same for both Game 1 and Game 2.
3. Make sure you fill out the Markdown ("?") with your game as well as the solution ("?").
4. Remember, **do not return the payoff**, return the strategy indices (a list of them).

## Before you code...

Solve the following game by hand using SEDS and weakly dominated strategies. 
The game has three (pure) Nash Equilibriums. 
You should find all of them.
This will help you think about what you need to implement to make the algorithm work.
**Hint**: You will need State Space Search from Module 1 and SEDS from Module 5 to get the full algorithm to work.

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 1/0 | 3/1 | 1/1 |
|1  | 1/1 | 3/0 | 0/1 |
|2  | 2/2 | 3/3 | 0/2 |

**Solutions**: 

If player 1 uses 0, player 2's best strategy is 1 or 2

If Player 1 uses 1, player 2's best strategy is either 0 or 2

If player 1 uses 2, player 2's best strategy is 1

player 2's best consistent strategy is 1. However we cannot remove any of these  via SEDS.

If player 2 uses 0, player 1's best strategy is 2

If player 2 uses 1, player 1's best strategy is 2

If player 2 uses 2, player 1's best strategy is 2

Player 1's best stratgy is 2, again however SEDS Cannot eliminate anything.

We can then state space search the entire grid as we could not return anything and we get these solutions:

(0, 1), (0, 2), (2, 1)

With SEWDS we would be able to eliminate a lot:

We would start by eliminating row 1 as row 0 weakly dominates it,

However nethier of the other two rows can weakly dominate the other so we are lft with only row 0 and 2,

Then we can eliminate column 0 as columns 1 dominates it, column 1 also dominates column 2.

We are then left with (0,1) and (2, 1) as the only spaces left to check and both are equilibriums

### Test Game 1. Create a 3x3 two player game

**that can only be solved using the Successive Elimintation of Strongly Dominated Strategies**


| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | (4,2) | (3,5) | (2,5) |
|1  | (2,3) | (1,1) | (1,1) |
|2  | (0,4) | (2,1) | (3,2) |

**Solution:** 

With SEDS we can start looking at each row.

First we can eliminate row 1 and as it is strictly dominated by row 0. Neither of the other rows can be eliminated as they are not strictly dominated.

So looking at columns, which none can be eliminated from. Thus we have to state space search and we find that (0, 1) is a solution.

With SEWDS we see this:

Row 1 is once again dominated and eliminated.

With columns however we see that column 1 is eliminated as it is weakly dominated by column 2, thus eliminating our solution to this game.

(0, 1)

In [20]:
# New test game designed for SEDS-only solvability
test_game_1 = [
    [(4, 2), (3, 5), (2, 5)],
    [(2, 3), (1, 1), (1, 1)],
    [(0, 4), (2, 1), (3, 2)]
]

solution = solve_game(test_game_1)
print(solution)

[(0, 1)]


In [21]:
assert solution == [(0, 1)] # insert your solution from above.

### Test Game 2. Create a 3x3 two player game

**that can only be solved using the Successive Elimintation of Weakly Dominated Strategies**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | (3,2) | (2,4) | (4,3) |
|1  | (4,3) | (1,1) | (3,2) |
|2  | (2,4) | (2,3) | (3,1) ||

**Solution:** (0,1), (1,0)

With SEWDS:

First we look at the rows, eliminate row 2 as it is dominated by row 0, no other rows can be eliminated.

Then we move onto columns, No columns can be removed leaving us to state space search finding (0,1), (1,0).

SEDS:

Cannot remove any rows or columns, meaning only state space search would find something which is not really SEDS as it is still the entire game board.

In [22]:
test_game_2 =[[(3, 2), (2, 4), (4, 3)],
    [(4, 3), (1, 1), (3, 2)],
    [(2, 4), (2, 3), (3, 1)]]

strong_solution = solve_game(test_game_2)
weak_solution = solve_game(test_game_2, weak=True)

In [14]:
assert strong_solution == []
assert weak_solution == [(0,1), (1,0)] # insert your solution from above.

### Test Game 3. Create a 3x3 two player game

**that cannot be solved using the Successive Elimintation of Dominated Strategies at all**

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | (3,3) | (1,2) | (1,1) |
|1  | (2,1) | (2,2) | (0,3) |
|2  | (1,1) | (3,0) | (2,2) |

**Solution:** None

In [23]:
test_game_3 = [[(3,3), (1,2), (1,1)], [(2,1), (2,2), (0,3)], [(1,1), (3,0), (2,2)]]

strong_solution = solve_game( test_game_3)
weak_solution = solve_game( test_game_3, weak=True)

In [24]:
assert strong_solution == []
assert weak_solution == []

### Test Game 4. Multiple Equilibria

You solve the following game by hand, above.
Now use your code to solve it.

| Player 1 / Player 2  | 0 | 1 | 2 |
|----|----|----|----|
|0  | 1/0 | 3/1 | 1/1 |
|1  | 1/1 | 3/0 | 0/1 |
|2  | 2/2 | 3/3 | 0/2 |

**Solutions:** (0, 1), (0, 2), (2, 1), but (0,2) cannot be found via SEDS or SEWDS, only with State Space Search, atleast that I could figure out

In [17]:
test_game_4 = [
[(1, 0), (3, 1), (1, 1)],
[(1, 1), (3, 0), (0, 1)],
[(2, 2), (3, 3), (0, 2)]]

strong_solution = solve_game( test_game_4)
weak_solution = solve_game( test_game_4, weak=True)

In [18]:
assert strong_solution == []
assert weak_solution == [(0, 1), (2, 1)] # put solution here

## Before You Submit...

1. Did you provide output exactly as requested? **Don't forget to fill out the Markdown tables with your games**.
2. Did you re-execute the entire notebook? ("Restart Kernel and Rull All Cells...")
3. If you did not complete the assignment or had difficulty please explain what gave you the most difficulty in the Markdown cell below.
4. Did you change the name of the file to `jhed_id.ipynb`?

Do not submit any other files.

## Difficulty

To be honest, this was the most difficult project for me so far. I don't specifically know why but I could not grasp SEDS or SEWDS for a very long part of this project and felt like I was just guessing things to work. Even now while I completed the project I am unsure if I actually did it in the correct manner. I think I grasp both SEDS and SEWDS well enough. The biggest issue for me was the initial problem. At this point I am fairly certain that you can only find (0,2) with state space search and not SEDS or SEWDS. I personally could not find a path that either algorithm would have found (0,2) as I would always eliminate it, however if there was a way I would love to know it as I think that hindered me the most. Very interesting project and module however, honestly quite fun (especially with the tic tac toe stuff for the Self Check).